# Introdução ao pacote de inversão

## Regressão linear básica

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(5, 20, 100)
a, b = 0.1, 500
y = a*x + b

In [ ]:
type(x)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.plot(x, y, '.k')

In [ ]:
from fatiando import utils

In [ ]:
y_real = utils.contaminate(y, 0.2)

In [ ]:
plt.plot(x, y_real, '.k')

In [ ]:
from fatiando.inversion import Misfit

In [ ]:
class Regressao(Misfit):
    
    def __init__(self, x, y):
        super(Regressao, self).__init__(data=y, nparams=2, islinear=True)
        self.x = x
        
    def predicted(self, p):
        a, b = p
        return a*self.x + b
    
    def jacobian(self, p):
        A = np.empty((self.ndata, self.nparams))
        A[:, 0] = self.x
        A[:, 1] = 1
        return A

In [ ]:
reg = Regressao(x, y_real)

In [ ]:
reg

In [ ]:
reg.fit()

In [ ]:
reg.estimate_

In [ ]:
reg.predicted()

In [ ]:
plt.plot(x, y_real, '.k')
plt.plot(x, reg.predicted(), '-r')

## Desafio: Implemente uma classe que faz o ajuste de uma parábola

In [ ]:
a, b, c = 0.01, 0.1, 100
y2 = a*x**2 + b*x + c
y2_real = utils.contaminate(y2, 0.1)

In [ ]:
plt.plot(x, y2, '.k')

In [ ]:
class Parabola(Misfit):
    
    def __init__(self, x, y):
        super(Parabola, self).__init__(data=y, nparams=3, islinear=True)
        self.x = x
        
    def predicted(self, p):
        a, b, c = p
        return a*self.x**2 + b*self.x + c
    
    def jacobian(self, p):
        A = np.empty((self.ndata, self.nparams))
        A[:, 0] = self.x**2
        A[:, 1] = self.x
        A[:, 2] = 1
        return A

In [ ]:
par = Parabola(x, y2_real).fit()
par.estimate_

In [ ]:
plt.plot(x, y2_real, '.k')
plt.plot(x, par.predicted(), '-r')

## Optimização não-linear

In [ ]:
par.config

In [ ]:
par.config('newton', initial=[0, 0, 0])

In [ ]:
par.fit()

In [ ]:
par.estimate_

In [ ]:
par.stats_

In [ ]:
par.config('acor', bounds=[0, 1, 0, 1, 0, 1000]).fit()

In [ ]:
par.estimate_

## Desafio: Estimar o epicentro

In [ ]:
receps = np.array([[1, 2], [4, 8], [10, 5]])
epicentro = [5, 5]

In [ ]:
def diferenca_tempo(epicentro, receptores, vp, vs):
    xe, ye = epicentro
    xr, yr = receptores.T
    t = (1/vs - 1/vp)*np.sqrt((xr - xe)**2 + (yr - ye)**2)
    return t

In [ ]:
plt.plot(receps[:, 0], receps[:, 1], '^b')
plt.plot(epicentro[0], epicentro[1], '*y')
plt.xlim(0, 15)
plt.ylim(0, 15)

In [ ]:
vp, vs = 4000, 4000/1.73
tempo = diferenca_tempo(epicentro, receps, vp, vs)
tempo_ruido = utils.contaminate(tempo, 0.01, percent=True)
tempo_ruido

In [ ]:
class Epicentro(Misfit):
    
    def __init__(self, receptores, tempos, vp, vs):
        super(Epicentro, self).__init__(data=tempos, nparams=2, islinear=False)
        self.vp = vp
        self.vs = vs
        self.alpha = (1/vs - 1/vp)
        self.receptores = receptores
        
    def predicted(self, p):        
        return diferenca_tempo(p, self.receptores, self.vp, self.vs)
    
    def jacobian(self, p):
        xe, ye = p
        xr, yr = self.receptores.T
        A = np.zeros((self.ndata, self.nparams))
        sqrt = np.sqrt((xr - xe)**2 + (yr - ye)**2)
        A[:, 0] = -self.alpha*(xr - xe)/sqrt
        A[:, 1] = -self.alpha*(yr - ye)/sqrt
        return A

In [ ]:
epic = Epicentro(receps, tempo_ruido, vp, vs)

In [ ]:
epic.config('levmarq', initial=[0, 0]).fit()

In [ ]:
epic.estimate_

In [ ]:
epic.config('steepest', initial=[0, 0]).fit()
epic.estimate_

In [ ]:
epic.config('acor', bounds=[1, 20, 1, 20]).fit()
epic.estimate_